# Supernote Index

## Model

In [2]:
from pathlib import Path
import logging
import sys
import yaml
import os

_LOGGER = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

SECRETS = Path("secrets.yaml")
MODEL_ID = 'gemini-1.5-flash'
EMBED_MODEL = "models/text-embedding-004"

secrets = yaml.safe_load(SECRETS.read_text())
os.environ["GOOGLE_API_KEY"] = secrets["gemini_api_key"]

## Llama index

In [10]:
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

_LOGGER.info("Initializing client library")
embed_model = GeminiEmbedding(
    model_name=EMBED_MODEL, api_key=secrets["gemini_api_key"], title="this is a document"
)
llm = Gemini()

INFO:__main__:Initializing client library


In [13]:
import llama_index
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

DOCS_DIR = Path("docs/")

Settings.llm = llm
Settings.embed_model = embed_model

_LOGGER.info("Loading documents")
documents = SimpleDirectoryReader(str(DOCS_DIR)).load_data()

INFO:__main__:Loading documents
DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 157
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/docs/Daily-00-P20231220073139675669IR1ax1C4F76D.yaml
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/docs/Daily-01-P20231221174744211995ocFlwfaIBnmC.yaml
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/docs/Daily-02-P20231225082735653731jvI1lbwRBdvP.yaml
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/docs/Daily-03-P20231228115558959584c8vxdTne3QLq.yaml
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/docs/Daily-04-P20240101115536493690nGWLtEW888wg.yaml
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/docs/Daily-05-P20240104105051877289y6uyEHXmdbt4.yaml
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/docs/Daily-06-P20240106144638872792NW8MkTyI2MNq.yaml
DEBUG:fsspec.local:open 

NameError: name 'service_context' is not defined

In [15]:
_LOGGER.info("Indexing documents")
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

INFO:__main__:Indexing documents
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ---
filename: Daily-00-P20231220073139675669IR1...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: status: null
  label: null
  critical: null
  d...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ---
filename: Daily-01-P20231221174744211995ocF...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ---
filename: Daily-02-P20231225082735653731jvI...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ---
filename: Daily-03-P20231228115558959584c8v...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ---
filename: Daily-04-P20240101115536493690nGW...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ---
filename: Daily-05-P20240104105051877289y6u...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ---
filename: Daily-06-P20240106144638872792NW8...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: -

In [21]:
query_engine = index.as_query_engine()
response = query_engine.query("Look at my goals from July and Auguest and help me set goals for September")
print("---")
print(response)

DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node ffd6be0a-cf7e-4bf5-9486-a7104ef24c93] [Similarity score:             0.70102] ---
filename: Monthly-09-P20240101140855582557jswAAktDnX1W.png
created_at: '2024-01-01T14:08:55.5...
> [Node f959e0de-c0cf-4cdb-af8a-cdf8bd3a5d10] [Similarity score:             0.684568] ---
filename: Monthly-39-P20240730073823816738mGeWOzgPBh6v.png
created_at: '2024-07-30T07:38:23.8...
---
In July, you reflected on your objectives and relationships, focusing on spending time with your children and grandparents. You also made progress on your work goals, including GDM plans, XPA life understanding, and synthetic data. You were also thinking about streamlining and decluttering your work. In August, you focused on meaningful connections with your children and supporting their interests. You also reflected on your priorities and how to invest in meaningful connections and supporting others. Based on these reflections, you might consider setting goals f

In [ ]:
# Persist the index
index.storage_context.persist()

In [20]:
# Example of persisting data

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/storage/docstore.json
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/storage/index_store.json
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/storage/graph_store.json
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/storage/default__vector_store.json
DEBUG:fsspec.local:open file: /Users/allen/Development/supernote-llm/storage/image__vector_store.json
